In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

engine

Engine(sqlite+pysqlite:///:memory:)

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2022-01-20 18:22:15,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:22:15,118 INFO sqlalchemy.engine.Engine select 'hello world'
2022-01-20 18:22:15,119 INFO sqlalchemy.engine.Engine [generated in 0.00271s] ()
[('hello world',)]
2022-01-20 18:22:15,120 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
with engine.connect() as conn:
    conn.execute(text("create table some_table (x int, y int)"))
    conn.execute(
        text("insert into some_table (x, y) values (:x, :y)"),
        [{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]
    )
    conn.commit()

2022-01-20 18:22:15,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:22:15,129 INFO sqlalchemy.engine.Engine create table some_table (x int, y int)
2022-01-20 18:22:15,130 INFO sqlalchemy.engine.Engine [generated in 0.00262s] ()
2022-01-20 18:22:15,131 INFO sqlalchemy.engine.Engine insert into some_table (x, y) values (?, ?)
2022-01-20 18:22:15,132 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ((1, 2), (2, 4))
2022-01-20 18:22:15,132 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2022-01-20 18:22:15,139 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:22:15,140 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-01-20 18:22:15,141 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ((6, 8), (9, 10))
2022-01-20 18:22:15,142 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        print(f'x: {row.x}, y: {row.y}')

2022-01-20 18:22:15,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:22:15,149 INFO sqlalchemy.engine.Engine select x, y from some_table
2022-01-20 18:22:15,150 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ()
x: 1, y: 2
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2022-01-20 18:22:15,151 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

metadata_obj

MetaData()

In [7]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    'user_account',
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

user_table

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [8]:
from sqlalchemy import ForeignKey

address_table = Table(
    'address',
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)

address_table

Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>, nullable=False), Column('email_address', String(), table=<address>, nullable=False), schema=None)

In [9]:
metadata_obj.create_all(engine)

2022-01-20 18:22:15,174 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:22:15,175 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-01-20 18:22:15,175 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:22:15,176 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-01-20 18:22:15,176 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:22:15,177 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-01-20 18:22:15,177 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:22:15,178 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-01-20 18:22:15,178 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:22:15,179 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-01-20 18:22:15,180 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2022-01-20 18:22:15,181 INFO sqlalchemy.engine.Engine 
C

In [10]:
from sqlalchemy.orm import registry

mapper_registry = registry()

Base = mapper_registry.generate_base()

from sqlalchemy.orm import relationship

Base

sqlalchemy.orm.decl_api.Base

In [11]:
class User(Base):
    __tablename__ = 'user_account'

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

    addresses = relationship('Address', back_populates='user')

    def __repr__(self):
        return f'User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})'

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))

    user = relationship('User', back_populates='addresses')

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [12]:
sandy = User(name='Sandy', fullname='Sandy Cheeks')

sandy

User(id=None, name='Sandy', fullname='Sandy Cheeks')

In [13]:
Base.metadata.create_all(engine)

2022-01-20 18:23:44,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:23:44,921 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-01-20 18:23:44,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:23:44,922 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-01-20 18:23:44,923 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:23:44,924 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
some_table = Table('some_table', metadata_obj, autoload_with=engine)

2022-01-20 18:24:31,677 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:24:31,678 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-01-20 18:24:31,679 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:24:31,681 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-20 18:24:31,681 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-01-20 18:24:31,683 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-01-20 18:24:31,683 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:24:31,684 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-01-20 18:24:31,684 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 18:24:31,685 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [15]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

In [16]:
from sqlalchemy import insert

stmt = insert(user_table).values(name='spongebob', fullname='Spongebob Squarepants')

stmt

In [17]:
compiled = stmt.compile()

compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [18]:
with engine.begin() as conn:
    result = conn.execute(stmt)

2022-01-20 18:28:41,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:28:41,663 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-01-20 18:28:41,663 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ('spongebob', 'Spongebob Squarepants')
2022-01-20 18:28:41,664 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {'name': 'sandy', 'fullname': 'Sandy Cheeks'},
            {'name': 'patrick', 'fullname': 'Patrick Star'}
        ]
    )
    conn.commit()

2022-01-20 19:00:10,139 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 19:00:10,140 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-01-20 19:00:10,140 INFO sqlalchemy.engine.Engine [generated in 0.00177s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-01-20 19:00:10,141 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
from sqlalchemy import select, bindparam

select_stmt = select(user_table.c.id, user_table.c.name + '@aol.com')

insert_stmt = insert(address_table).from_select(
    ['user_id', 'email_address'], select_stmt
)

print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [21]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)

print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [22]:
insert_stmt = insert(address_table).from_select(
    ['user_id', 'email_address'], select_stmt
)

print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


In [23]:
stmt = select(user_table).where(user_table.c.name == 'spongebob')

print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [25]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-01-20 19:07:37,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 19:07:37,756 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-01-20 19:07:37,757 INFO sqlalchemy.engine.Engine [generated in 0.00182s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-01-20 19:07:37,758 INFO sqlalchemy.engine.Engine ROLLBACK


In [26]:
stmt = select(User).where(User.name == 'spongebob')

from sqlalchemy.orm import Session

with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-01-20 19:11:46,923 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 19:11:46,925 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-01-20 19:11:46,926 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-01-20 19:11:46,928 INFO sqlalchemy.engine.Engine ROLLBACK


In [27]:
print(select(user_table))
print(select(user_table.c.name, user_table.c.fullname))
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [28]:
row = session.execute(select(User)).first()

2022-01-20 19:13:38,164 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-01-20 19:13:38,165 INFO sqlalchemy.engine.Engine [cached since 3.763s ago] ()


In [29]:
row[0]

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [30]:
session.execute(select(Address)).all()

2022-01-20 19:15:47,224 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address
2022-01-20 19:15:47,225 INFO sqlalchemy.engine.Engine [cached since 7.07s ago] ()


[]

In [31]:
with engine.begin() as conn:
    conn.execute(insert_stmt)

2022-01-20 19:16:10,187 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 19:16:10,190 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account
2022-01-20 19:16:10,191 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('@aol.com',)
2022-01-20 19:16:10,191 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
session.execute(select(Address)).all()

2022-01-20 19:16:20,637 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address
2022-01-20 19:16:20,638 INFO sqlalchemy.engine.Engine [cached since 40.48s ago] ()


[(Address(id=1, email_address='spongebob@aol.com'),),
 (Address(id=2, email_address='sandy@aol.com'),),
 (Address(id=3, email_address='patrick@aol.com'),)]

In [33]:
session.execute(
    select(User.name, Address).
    where(User.id == Address.user_id).
    order_by(Address.id)
).all()

2022-01-20 19:17:45,374 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2022-01-20 19:17:45,375 INFO sqlalchemy.engine.Engine [cached since 1.997s ago] ()


[('spongebob', Address(id=1, email_address='spongebob@aol.com')),
 ('sandy', Address(id=2, email_address='sandy@aol.com')),
 ('patrick', Address(id=3, email_address='patrick@aol.com'))]

In [34]:
from sqlalchemy import func, cast

stmt = select(
    ("Username: " + user_table.c.name).label("username"),
).order_by(user_table.c.name)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f'{row.username}')

2022-01-20 19:20:22,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 19:20:22,362 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-01-20 19:20:22,362 INFO sqlalchemy.engine.Engine [generated in 0.00235s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-01-20 19:20:22,364 INFO sqlalchemy.engine.Engine ROLLBACK


In [35]:
squidward = User(name="squidward", fullname="Squidward Tentacles")

session.add(squidward)

session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles')])

In [36]:
session.flush()

2022-01-20 20:52:52,403 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-01-20 20:52:52,405 INFO sqlalchemy.engine.Engine [generated in 0.00203s] ('squidward', 'Squidward Tentacles')


In [37]:
squidward.id

4

In [38]:
some_squidward = session.get(User, 4)

some_squidward

User(id=4, name='squidward', fullname='Squidward Tentacles')

In [39]:
session.commit()

2022-01-20 20:54:06,282 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
sandy = session.execute(select(User).filter_by(name='sandy')).scalar_one()

sandy

2022-01-20 20:55:38,472 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-01-20 20:55:38,474 INFO sqlalchemy.engine.Engine [cached since 2306s ago] ('sandy',)


User(id=2, name='sandy', fullname='Sandy Cheeks')

In [41]:
sandy.fullname = "Sandy Squirrel"

sandy in session.dirty

True

In [42]:
session.flush()

2022-01-20 20:57:47,647 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2022-01-20 20:57:47,648 INFO sqlalchemy.engine.Engine [generated in 0.00155s] ('Sandy Squirrel', 2)


In [43]:
sandy_fullname = session.execute(
    select(User.fullname).where(User.id == 2)
).scalar_one()

2022-01-20 20:59:16,964 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2022-01-20 20:59:16,965 INFO sqlalchemy.engine.Engine [cached since 4.117s ago] (2,)


In [44]:
sandy_fullname

'Sandy Squirrel'

In [45]:
sandy in session.dirty

False

In [47]:
from sqlalchemy import update

session.execute(
    update(User)
    .where(User.name == 'sandy')
    .values(fullname='Sandy Squirrel Extraordinare')
)

2022-01-20 21:01:36,560 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2022-01-20 21:01:36,562 INFO sqlalchemy.engine.Engine [cached since 23.03s ago] ('Sandy Squirrel Extraordinare', 'sandy')


In [48]:
sandy.fullname

'Sandy Squirrel Extraordinare'

In [49]:
partrick = session.get(User, 3)

session.delete(partrick)

2022-01-20 21:02:13,654 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-01-20 21:02:13,656 INFO sqlalchemy.engine.Engine [generated in 0.00145s] (3,)


In [67]:
session.rollback()

In [68]:
partrick

User(id=3, name='patrick', fullname='Patrick Star')

In [69]:
patrick_addr = session.execute(
    select(Address)
    .where(Address.user_id == partrick.id)
).scalar_one()

session.delete(patrick_addr)
session.delete(partrick)

2022-01-20 21:13:00,881 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE address.user_id = ?
2022-01-20 21:13:00,882 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (3,)


In [70]:
session.flush()

2022-01-20 21:13:05,042 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-01-20 21:13:05,043 INFO sqlalchemy.engine.Engine [cached since 594.8s ago] (3,)
2022-01-20 21:13:05,045 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-01-20 21:13:05,046 INFO sqlalchemy.engine.Engine [generated in 0.00061s] (3,)
2022-01-20 21:13:05,046 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2022-01-20 21:13:05,047 INFO sqlalchemy.engine.Engine [generated in 0.00071s] (3,)


In [71]:
session.execute(
    select(User)
        .where(User.name == 'patrick')
).first()

2022-01-20 21:13:14,742 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-01-20 21:13:14,743 INFO sqlalchemy.engine.Engine [cached since 3362s ago] ('patrick',)


In [72]:
partrick in session

False

In [73]:
sandy in session

True

In [75]:
sandy.addresses

2022-01-20 21:14:05,393 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-01-20 21:14:05,394 INFO sqlalchemy.engine.Engine [cached since 655.2s ago] (2,)


[Address(id=2, email_address='sandy@aol.com')]

In [76]:
sandy.fullname

'Sandy Cheeks'

In [77]:
session.rollback()

2022-01-20 21:16:57,749 INFO sqlalchemy.engine.Engine ROLLBACK


In [78]:
sandy.fullname

'Sandy Cheeks'

In [79]:
partrick in session

True

In [80]:
u1 = User(name = 'pkrabs', fullname='Pearl Krabs')

u1.addresses

[]

In [81]:
a1 = Address(email_address='pearl.krabs@gmail.com')

u1.addresses.append(a1)

u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com')]

In [82]:
a1.user

User(id=None, name='pkrabs', fullname='Pearl Krabs')

In [83]:
a2 = Address(email_address="pearl@aol.com", user=u1)

In [84]:
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com'),
 Address(id=None, email_address='pearl@aol.com')]

In [85]:
session.add(u1)

session.flush()

2022-01-20 21:21:06,247 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-01-20 21:21:06,248 INFO sqlalchemy.engine.Engine [cached since 1694s ago] ('pkrabs', 'Pearl Krabs')
2022-01-20 21:21:06,250 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-01-20 21:21:06,250 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('pearl.krabs@gmail.com', 5)
2022-01-20 21:21:06,251 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-01-20 21:21:06,251 INFO sqlalchemy.engine.Engine [cached since 0.001661s ago] ('pearl@aol.com', 5)


In [86]:
(u1.id, a1.user_id, a2.id)

(5, 5, 5)

In [87]:
session.commit()

2022-01-20 21:22:08,975 INFO sqlalchemy.engine.Engine COMMIT


In [88]:
u1.id

5

In [89]:
print(
    select(Address.email_address)
    .select_from(User)
    .join(User.addresses)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [90]:
print(
    select(Address.email_address)
    .join_from(User, Address)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [91]:
from sqlalchemy.orm import aliased

address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)

print(
    select(User)
    .join(User.addresses.of_type(address_alias_1))
    .where(address_alias_1.email_address=="patrick@gmail.com")
    .join(User.addresses.of_type(address_alias_2))
    .where(address_alias_2.email_address=="patrick@aol.com")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


In [92]:
stmt = (
    select(User.fullname)
    .join(User.addresses.and_(Address.email_address == 'pearl.krabs@gmail.com'))
)

session.execute(stmt).all()

2022-01-20 21:29:22,460 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account JOIN address ON user_account.id = address.user_id AND address.email_address = ?
2022-01-20 21:29:22,461 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ('pearl.krabs@gmail.com',)


[('Pearl Krabs',)]

In [93]:
session.execute(
    select(User.fullname)
    .where(User.addresses.any(Address.email_address == 'pearl.krabs@gmail.com'))
).all()

2022-01-20 21:30:53,082 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE EXISTS (SELECT 1 
FROM address 
WHERE user_account.id = address.user_id AND address.email_address = ?)
2022-01-20 21:30:53,084 INFO sqlalchemy.engine.Engine [generated in 0.00135s] ('pearl.krabs@gmail.com',)


[('Pearl Krabs',)]

In [94]:
session.execute(
    select(User.fullname)
    .where(~User.addresses.any())
).all()

2022-01-20 21:31:49,106 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE NOT (EXISTS (SELECT 1 
FROM address 
WHERE user_account.id = address.user_id))
2022-01-20 21:31:49,108 INFO sqlalchemy.engine.Engine [cached since 2.713s ago] ()


[('Squidward Tentacles',)]

In [95]:
session.execute(
    select(Address.email_address)
    .where(Address.user.has(User.name=='pkrabs'))
).all()

2022-01-20 21:33:41,144 INFO sqlalchemy.engine.Engine SELECT address.email_address 
FROM address 
WHERE EXISTS (SELECT 1 
FROM user_account 
WHERE user_account.id = address.user_id AND user_account.name = ?)
2022-01-20 21:33:41,145 INFO sqlalchemy.engine.Engine [cached since 1.424s ago] ('pkrabs',)


[('pearl.krabs@gmail.com',), ('pearl@aol.com',)]